In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import pandas as pd
import numpy as np
from gtda.time_series import SlidingWindow
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.keras.backend import set_session
config = tf.compat.v1.ConfigProto() 
config.gpu_options.allow_growth = True  
config.log_device_placement = True  
sess2 = tf.compat.v1.Session(config=config)
set_session(sess2) 
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.backend import sigmoid
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
import tensorflow_datasets as tfds
from tensorflow.keras.models import load_model
import tensorflow_hub as hub

import get_dataset as kws_data
import kws_util
import argparse

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:21:00.0, compute capability: 8.6



## Import Dataset

In [2]:
Flags, unparsed = kws_util.parse_command()
Flags.window_size_ms=40.0
Flags.window_stride_ms=20.0
Flags.data_dir = '/home/nesl/209as_sec/audio_ks/data'
print('We will download data to {:}'.format(Flags.data_dir))
ds_train, ds_test, ds_val = kws_data.get_training_data(Flags)
print("Done getting data")
train_shuffle_buffer_size = 85511
val_shuffle_buffer_size = 10102
test_shuffle_buffer_size = 4890

ds_train = ds_train.shuffle(train_shuffle_buffer_size)
ds_val = ds_val.shuffle(val_shuffle_buffer_size)
ds_test = ds_test.shuffle(test_shuffle_buffer_size)

ds_train = ds_train.map(lambda x, y: (x, tf.one_hot(y, depth=12)))
ds_val = ds_val.map(lambda x, y: (x, tf.one_hot(y, depth=12)))
ds_test = ds_test.map(lambda x, y: (x, tf.one_hot(y, depth=12)))

We will download data to /home/nesl/209as_sec/audio_ks/data
Done getting data


## Train Model

In [3]:
inputs = layers.Input(shape=(49, 10, 1))
x = inputs 
x = layers.Reshape((49,10))(x)
x = layers.GRU(400)(x)
outputs = layers.Dense(12, activation="softmax", name="pred")(x)
model = Model(inputs = inputs, outputs = outputs)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 49, 10, 1)]       0         
_________________________________________________________________
reshape (Reshape)            (None, 49, 10)            0         
_________________________________________________________________
gru (GRU)                    (None, 400)               494400    
_________________________________________________________________
pred (Dense)                 (None, 12)                4812      
Total params: 499,212
Trainable params: 499,212
Non-trainable params: 0
_________________________________________________________________


In [4]:
model_name = 'gru_large.h5'
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

checkpoint = ModelCheckpoint(model_name, save_best_only=True, monitor='val_accuracy')
model.fit(ds_train,
              epochs=30,
              validation_data=ds_val,
              callbacks=[checkpoint],
              shuffle=True,
              verbose=1)

Epoch 1/30
856/856 [==============================] - 19s 7ms/step - loss: 0.4664 - accuracy: 0.8598 - val_loss: 0.2003 - val_accuracy: 0.9392
Epoch 2/30
856/856 [==============================] - 17s 7ms/step - loss: 0.1796 - accuracy: 0.9437 - val_loss: 0.1613 - val_accuracy: 0.9513
Epoch 3/30
856/856 [==============================] - 17s 7ms/step - loss: 0.1325 - accuracy: 0.9587 - val_loss: 0.1492 - val_accuracy: 0.9534
Epoch 4/30
856/856 [==============================] - 17s 7ms/step - loss: 0.1042 - accuracy: 0.9670 - val_loss: 0.1358 - val_accuracy: 0.9599
Epoch 5/30
856/856 [==============================] - 17s 7ms/step - loss: 0.0853 - accuracy: 0.9726 - val_loss: 0.1535 - val_accuracy: 0.9542
Epoch 6/30
856/856 [==============================] - 17s 7ms/step - loss: 0.0698 - accuracy: 0.9778 - val_loss: 0.1498 - val_accuracy: 0.9574
Epoch 7/30
856/856 [==============================] - 17s 7ms/step - loss: 0.0594 - accuracy: 0.9812 - val_loss: 0.1602 - val_accuracy: 0.9550

## Evaluate Model

In [5]:
new_model = load_model(model_name)
new_model.evaluate(ds_test)

49/49 [==============================] - 2s 11ms/step - loss: 0.3512 - accuracy: 0.9221


[0.35124754905700684, 0.9220858812332153]